In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd "/content/drive/MyDrive/Python Practical lab/SGP"

/content/drive/MyDrive/Python Practical lab/SGP


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation

In [ ]:
batchsize=8

In [ ]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory(r'dataset/train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'dataset/train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 1975 images belonging to 4 classes.
Found 492 images belonging to 4 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'dataset/test',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 433 images belonging to 4 classes.


In [ ]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(4,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [ ]:
checkpoint = ModelCheckpoint(r'models/model.h5',monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3 )

callbacks=[checkpoint,earlystop,learning_rate]

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,validation_data=validation_data,validation_steps=validation_data.samples//batchsize,callbacks=callbacks,epochs=5)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
246/246 [==============================] - 963s 4s/step - loss: 0.7011 - accuracy: 0.6924 - val_loss: 0.4291 - val_accuracy: 0.7439

Epoch 00001: val_loss improved from inf to 0.42912, saving model to models/model.h5
Epoch 2/5
246/246 [==============================] - 24s 98ms/step - loss: 0.4688 - accuracy: 0.7631 - val_loss: 0.4246 - val_accuracy: 0.7643

Epoch 00002: val_loss improved from 0.42912 to 0.42456, saving model to models/model.h5
Epoch 3/5
246/246 [==============================] - 24s 96ms/step - loss: 0.4218 - accuracy: 0.7911 - val_loss: 0.4117 - val_accuracy: 0.7561

Epoch 00003: val_loss improved from 0.42456 to 0.41170, saving model to models/model.h5
Epoch 4/5
246/246 [==============================] - 24s 97ms/step - loss: 0.4178 - accuracy: 0.7783 - val_loss: 0.4276 - val_accuracy: 0.7029

Epoch 00004: val_loss did not improve from 0.41170
Epoch 5/5
246/246 [==============================] - 24s 96ms/step - loss: 0.3967 - accuracy: 0.7885 - val_loss: 0